In [24]:
import os
import zipfile
import pathlib
from pathlib import Path
import kaggle
from dotenv import load_dotenv

In [25]:
path = Path('us-patent-phrase-to-phrase-matching')
if not path.exists():
    kaggle.api.competition_download_cli(str(path))
    zipfile.ZipFile(f'{path}.zip').extractall(path)

100%|██████████| 682k/682k [00:00<00:00, 1.43GB/s]

In [26]:
import pandas as pd

df = pd.read_csv(path/'train.csv')
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [27]:
## Create the input 

df['input'] = df['context'] + ' ' + df['anchor'] + ' ' + df['target']
df[['input', 'score']].head()

,input,score
0,A47 abatement abatement of pollution,0.50
1,A47 abatement act of abating,0.75
2,A47 abatement active catalyst,0.25
3,A47 abatement eliminating process,0.50
4,A47 abatement forest region,0.00


In [29]:
!pip install datasets transformers
from datasets import Dataset, DatasetDict

ds = Dataset.from_pandas(df[['input', 'score']])
ds

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.1/75.1 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.7/119.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.3/150.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.9/193.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.4/242.4 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.6/221.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 32.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.12.0
    Uninstalling fsspec-2025.12.0:
      Successfully uninstalled fsspec-2025.12.0


Dataset({
    features: ['input', 'score'],
    num_rows: 36473
})

In [30]:
model_name = "distilbert-base-uncased"

from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [31]:
print(df['input'][0])
tokenizer.tokenize(df['input'][0])

A47 abatement abatement of pollution


['a',
 '##47',
 'aba',
 '##tem',
 '##ent',
 'aba',
 '##tem',
 '##ent',
 'of',
 'pollution']

In [32]:
tokenizer.tokenize('My name is jeff')

['my', 'name', 'is', 'jeff']

In [33]:
# Function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['input'], padding="max_length", truncation=True)  

In [34]:
tokenized_ds = ds.map(tokenize_function, batched=True)
tokenized_ds

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

Dataset({
    features: ['input', 'score', 'input_ids', 'attention_mask'],
    num_rows: 36473
})

In [35]:
tokenized_ds = tokenized_ds.rename_columns({'score':'labels'})

In [36]:
eval_df = pd.read_csv(path/'test.csv')
eval_df['input'] = eval_df.context + ' ' + eval_df.target + ' ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tokenize_function, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [37]:
# metric for evaluation
def corr_d(eval_pred):
    import numpy as np
    from scipy.stats import pearsonr
    predictions, labels = eval_pred
    return {'pearson': pearsonr(labels, predictions[:,0])[0]}

In [43]:
from transformers import TrainingArguments,Trainer
batch_size = 60
epochs = 10

lr = 8e-5

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=lr,
    optim="adamw_torch",
    warmup_ratio=0.1,
    lr_scheduler_type="cosine",
    fp16=False,
    bf16=True,
    # evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.01,
    report_to="none")

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    eval_dataset=eval_ds,
    compute_metrics=corr_d,
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [44]:
trainer.train()

Step,Training Loss
500,0.060200
1000,0.032000
1500,0.021300
2000,0.017700
2500,0.016500


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

In [ ]:
import numpy as np

preds = trainer.predict(eval_ds).predictions.astype(float)
preds = np.clip(preds, 0, 1)
preds[:10]


In [ ]:
import datasets

submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)